# Processing Data

Earlier, we mentioned that most of our notebooks, and even some of our individual cells, follow a fairly standard pattern:

1. Read Data
2. Clean Data
3. Filter Data
4. **Process Data**
5. Output Data

This article will be about processing data.

It is not uncommon for a python notebook or cell to not actually do any data processing.  Sometimes, all you want to do is read, clean, and filter the data before output.  You may, for instance, want to read in a pcap file, clean it up, filter it down to just conversations involving a particular ip address, and then display the data, or possibly graph the payload size over time, etc.

In those cases, there is no explicit data processing stage.

When there is a data processing stage, I find that 9 times out of 10, it going to end up being a group by/aggregation operation.  Most transformations we have done to the data is to create summary reports on our data which involve aggregating things up. We may resample the data into larger time series chunks, or we may want to split our data up by network direction and calculate summary statistics.  Most of the transformations we do to the data after filtering is to create higher level summaries of the data.  And that falls generally to the `groupby` function and its relatives in the pandas library

One of the most powerful and commonly used features in Pandas is the `groupby` operation. This function essentially allows you to split your data into groups, apply a function to each group independently, and then combine the results into an output DataFrame.

Let's break it down a bit more:

### Split-Apply-Combine

The groupby operation follows what is known as the split-apply-combine pattern.

#### Split: 

First, data is split into groups based on some criteria. For instance, if we have a dataset with information about different cities in various countries, we could group our data by 'Country'. This will create a separate group for each unique country in our dataset.

#### Apply: 

After the data is split into groups, you then apply a function to each group. The function could be a standard aggregation function (like mean, min, max, sum, count, etc.), or a custom function defined by you. The function is applied independently to each group.

#### Combine: 

Finally, the resulting groups are combined back into a new DataFrame, with the group identifier as the index and the applied function results as the DataFrame values.

### Usage

Here's an example of how to use the groupby function in Pandas:

In [2]:
import pandas as pd

# create a simple dataframe
data = {
    'Country': ['USA', 'USA', 'Canada', 'Canada', 'USA', 'Canada'],
    'City': ['New York', 'Los Angeles', 'Toronto', 'Montreal', 'Chicago', 'Vancouver'],
    'Population': [8600000, 4000000, 2800000, 1700000, 2700000, 630000],
}

df = pd.DataFrame(data)

# group by country and find the sum of populations in each country
grouped = df.groupby('Country')['Population'].sum()

print(grouped)

Country
Canada     5130000
USA       15300000
Name: Population, dtype: int64


### Potential Issues

While groupby is an incredibly useful function, there are some potential pitfalls to keep in mind:

#### Missing values: 

If there are missing values in the columns you want to group by, they will be excluded from the groups. You may want to handle missing values before applying groupby.

#### Aggregation over non-numeric data: 

If you attempt to apply a numeric function (like mean, sum, etc.) over non-numeric data, you'll get an error. Make sure your data types are appropriate for the aggregation function you want to apply.

#### Large datasets: 

The groupby operation can become resource-intensive with very large datasets. If you notice a slowdown when using this function, consider optimizing your use of the groupby operation or employ other optimizations like working with a subset of your data or using different data types to save on memory.


apply() is another powerful Pandas method used alongside groupby() to perform complex operations on grouped data that can't be achieved with standard aggregation functions.

The apply() function applies a function along any axis of a DataFrame. When we use apply() with groupby(), we can apply any particular function to each group of values.

Here's how to use the apply() function:

In [3]:
import pandas as pd

# create a simple data frame
data = {
    'Country': ['USA', 'USA', 'Canada', 'Canada', 'USA', 'Canada'],
    'City': ['New York', 'Los Angeles', 'Toronto', 'Montreal', 'Chicago', 'Vancouver'],
    'Population': [8600000, 4000000, 2800000, 1700000, 2700000, 630000],
    'Area': [780, 1302, 630, 165, 589, 115]
}

df = pd.DataFrame(data)

# Define a function that calculates population density
def calculate_density(xdata):
    # Population density is population divided by area
    return xdata['Population'].sum() / xdata['Area'].sum()

grouped = df.groupby('Country').apply(calculate_density)
print(grouped)

Country
Canada    5637.362637
USA       5728.191689
dtype: float64


In this example, calculate_density function is applied to each group in the DataFrame separately, and the results are combined into a new DataFrame.

**Points to note:**

The function passed to apply() should expect a DataFrame (representing the data for a group) as its argument and return a Series, a DataFrame or a scalar.

apply() can be used to perform more complex operations than aggregation functions and transform(). However, it should be noted that apply() can be slower than the specialized functions (like transform() or agg()) and should be used when the operation cannot be performed by the others.

New users to Pandas often get confused by how to use apply() and groupby() together, so it's worth spending some time practicing with different examples to fully understand how they work together. Depending on the use case, the combination can provide a high level of flexibility.

#### Apply

The other operations that I find in the data processing section of a notebook/cell is the application of functions over rows, and merging/joining dataframes.

Lets look at applying a function over every row in a dataframe.

In [4]:
df

,Country,City,Population,Area
0,USA,New York,8600000,780
1,USA,Los Angeles,4000000,1302
2,Canada,Toronto,2800000,630
3,Canada,Montreal,1700000,165
4,USA,Chicago,2700000,589
5,Canada,Vancouver,630000,115


In this experiment, I am going to create a fifth column that contains the md5 hash of all the values in each row.  

Python has a hashlib that will do this, but it expects single strings/bytes, not an entire column of values.  So, we will have to apply the function to each row.

In [9]:
import hashlib

hashlib.md5("ASDFASDFASD".encode()).hexdigest()

'6a433feb54ba001ebc1f385ef9e9de84'

In [13]:
df.assign(hash = lambda df: df.apply(lambda r: hashlib.md5((r.Country + r.City + str(r.Population) + str(r.Area)).encode()).hexdigest(), axis='columns'))

,Country,City,Population,Area,hash
0,USA,New York,8600000,780,acc43c3a25694f9b72961483cf0ea5e5
1,USA,Los Angeles,4000000,1302,626b1e66a5ebf04321a66c459f49b73a
2,Canada,Toronto,2800000,630,68e8eba6b9f92295f191952fb514a90d
3,Canada,Montreal,1700000,165,e6710b15df4f8e2def5147bf8cd28c24
4,USA,Chicago,2700000,589,d9fe39ed1d90342cdedb185e34ea4d7f
5,Canada,Vancouver,630000,115,5b770e8a2152cfdb45e3aa05187a197f


The assign function, which we have seen before, accepts a function that takes a dataframe.  For many operations thats all you need, but in this case, we need to apply a function across every row.  This is done with the:

```python
df.assign(hash = lambda xf: xf.apply(fun, axis='columns'))
```

The assign function accepts a simple function that will return a series based on the xf dataframe passed in.  To generate this series, we use the xf.apply method, with a function we want to run and the axis over which we want to run it. This is a bit confusing, but you say axis='columns' when you want the function to process all the columns of a single row. You can pass axis='rows' if you want to apply the function over every column of a row.  I have never used axis='rows', only columns, and I expect that is true for most people. 

The function we pass to apply ends up being a function that receives a row, and can use all of the various columns on that row.

```python
lambda r: hashlib.md5(r.Country, r.City, ...)
```

is applied to each row of the dataframe, and the resulting values are put togeter in to a series that gets attached to hash via the assign statement.


#### Merge/Join

Merge and join are two important operations that are often done in data manipulation and analysis.

###### Pandas Merge

Pandas merge connects columns or indexes in DataFrame based on one or more keys. It provides a very flexible interface, allowing you to merge on indexes or columns and offers various types of set operations to use during the merge.

Here's the basic syntax:

```python
pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True)
```
* left: A DataFrame object.
* right: Another DataFrame object.
* on: Columns (names) to join on. Must be found in both the left and right DataFrame objects.
* left_on: Columns from the left DataFrame to use as keys.
* right_on: Columns from the right DataFrame to use as keys.
* left_index: If True, use the index (row labels) from the left DataFrame as its join key(s).
* right_index: Same usage as left_index for the right DataFrame.
* how: One of 'inner', 'outer', 'left', or 'right'. Default is 'inner'. Each method has certain attributes of set theory.

Here's an example:

```python
merged_df = pd.merge(df1, df2, on='id')
```

This will merge two dataframes, df1 and df2, based on the id column.

#### Join

Pandas DataFrame join is a convenient method for combining the columns of two DataFrames. This is quite similar to the merge method but the join method will be performed on the columns on the default index.

Here is a basic syntax of join:

```python
df.join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
```

* other: DataFrame, Series with name field set, or dict-like
* on: Column or index level names to join on in the DataFrame.
* how: How to handle the operation of the two objects. 1. 'left': Use keys from left frame only 2. 'right': Use keys from right frame only 3. 'outer': Use union of keys from both frames 4. 'inner': Use intersection of keys from both frames
* lsuffix: Suffix to use from left frame’s overlapping columns
* rsuffix: Suffix to use from right frame’s overlapping columns

Some general rules to remember:

You would use merge() when you want to merge on a particular column and you can specify this column.

You would use join() when you are joining on index or you want to join on the basis of the index of the right frame.

Keep in mind that in most cases, merge and join can be used interchangeably, it largely depends on the specific use-case and the layout of your data.

Lets do a more concrete example.

Lets take our city and country data and join it with a list of branch locations to work out the country and population of each branches area.

In [16]:
branches = pd.DataFrame({'Name': ['B1', 'B2', 'B3', 'B4'], 'City': ['New York', 'Los Angeles', 'Chicago', 'Vancouver']})
branches

,Name,City
0,B1,New York
1,B2,Los Angeles
2,B3,Chicago
3,B4,Vancouver


In [17]:
branches.merge(df, left_on='City', right_on='City', how='inner')

,Name,City,Country,Population,Area
0,B1,New York,USA,8600000,780
1,B2,Los Angeles,USA,4000000,1302
2,B3,Chicago,USA,2700000,589
3,B4,Vancouver,Canada,630000,115


And now we have all the branches coupled with their city, country, and pop all in one dataframe.

Joining and merging are fundamental operations in data manipulations and cleaning. Here are reasons why one would want to use join or merge pandas methods:

**Combining data:** You might have data spread across multiple sources and you need to bring it together. This is the most common use case.

**Missing Data:** You might want to join or merge dataframes based on specific columns to fill in the missing values in one DataFrame using the values from another DataFrame.  Or, by doing inner joins, you can exclude data in the second data frame that doesnt exist in the first.

**Comparing data frames:** Sometimes, we need compare two or more dataframes. In this case, join and merge comes in handy.  An inner join between two data frames will get you only the elements in common between two datasets.  Some outer joins will only get you the ones in one or the other dataframe, allowing basically set operations (intersection, etc) to be performed on dataframes.


For example, let's say you have one DataFrame that contains information about products for an e-commerce store like product_id, price, category, etc., and a second DataFrame that contains sales data like sale_id, product_id, quantity, customer, etc. If you wanted to find out how many items of each category were sold, you'd need to join these two data frames together on the product_id field.

It's important to note that the method you choose depends on the specifics of your situation and what you're trying to achieve. `merge()` is more versatile and can be used in almost any situation, but `join()` is simpler and can be more convenient when you want to combine DataFrames based on their indexes.

### Pivot Table

Its not as common used, but pivot table and just plain pivot turns out to be somewhat useful in pandas as well.

The pivot_table() function is used to create a spreadsheet-style pivot table as a DataFrame in Python pandas. The levels in the pivot table are stored in a MultiIndex object (hierarchical indexes) on the index and columns of the result DataFrame.

It may be easiest to understand this if we show an example.

In [18]:
import pandas as pd
import numpy as np

# Creating a sample DataFrame
data = {
    "City": ["NY", "NY", "LA", "LA", "NY", "LA"],
    "Temp": [60, 58, 75, 76, 59, 74],
    "Humidity": [30, 35, 25, 20, 37, 22],
    "Weather": ['Cloudy', 'Sunny', 'Sunny', 'Sunny', 'Rain', 'Rain']
}

df = pd.DataFrame(data)

df

,City,Temp,Humidity,Weather
0,NY,60,30,Cloudy
1,NY,58,35,Sunny
2,LA,75,25,Sunny
3,LA,76,20,Sunny
4,NY,59,37,Rain
5,LA,74,22,Rain


In [20]:
pivot = df.pivot_table(index='City', values=['Temp', 'Humidity'], aggfunc='mean')
pivot

,Humidity,Temp
City,,
LA,22.333333,75.0
NY,34.000000,59.0


We used the parameter index='City' to group the data by the City column.

We specified values=['Temp', 'Humidity'] to calculate the mean temperature and humidity in each city.

We used aggfunc='mean' to calculate the mean. If we wanted to calculate the total instead, we could have used 'sum'.

The pivot_table function above took a list of city temp and humidity values and twisted the table around until we had an index of unique cities, and in the temp and humidity columns we end up with a mean of the various measurements we calculated.

There is a slightly more general purpose version of this function called just `pivot`


#### Pivot

The pivot() function in pandas is used to reshape data (produce a "pivot" table) based on column values. It uses unique values from specified index/ columns to form axes of the resulting DataFrame.

Here is the function signature:

```python
DataFrame.pivot(self, index=None, columns=None, values=None)
```

Parameters:

index: string or object. This is the column in the dataframe that the pivoted data will use as its new index.
columns: string or object. The column whose values will become the new columns in the pivoted data.
values: string or object. The column to use for populating new frame’s values.

An example usage:

In [21]:
df = pd.DataFrame({
    'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
    'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
    'baz': [1, 2, 3, 4, 5, 6],
    'zoo': ['x', 'y', 'z', 'q', 'w', 't']
})

df.pivot(index='foo', columns='bar', values='baz')

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


Unlike pivot_table(), the pivot() function does not do any aggregation. It simply reshapes the data as described. If there are any duplicate entries for the index/column combinations, pivot will throw a ValueError.

When might one use pivot()?

You would use pivot() when you want to move one column's entries to new columns and fill them with corresponding values from another column, but no aggregation operation is needed. It is a simple reshaping of the DataFrame structure.

Problems to look out for:

Data Size: Pivotting is usually not a good idea if you have a large amount of data. It can significantly increase the amount of data in the DataFrame (since it makes a new column for each unique value), using up a lot of memory.

Duplicate Entries: As mentioned earlier pivoting requires the index/column combinations to be unique. If they are not, you will need to use some sort of aggregation (pivot_table() can do this) to combine the duplicate entries in some way.

Note: In general, if you have any duplicate values in the columns specified for the 'index' or 'columns' parameters of the pivot() function, it's better to use the pivot_table() function. Otherwise, if your data is clean and well structured without duplicate rows, using the pivot() function is usually faster than applying pivot_table().

I often find it useful to use pivot when I want to rotate a series of timeseries values where each row is a time, type, and value, into a multi column dataset where each type gets its own column.

In [26]:
import pandas as pd
import numpy as np

# Create a date range with hourly frequency
date_range = pd.date_range(start='1/1/2022', end='2/7/2022', freq='H')

# Create a column with values from the set ('temp', 'humidity', 'windchill')
weather = np.random.choice(['temp', 'humidity', 'windchill'], size=len(date_range))

# Create a column with random values between 0 to 100
values = np.random.randint(0, 100, size=len(date_range))

# Construct the dataframe
df = pd.DataFrame({
    'Time': date_range,
    'Weather': weather,
    'Values': values
})

df

,Time,Weather,Values
0,2022-01-01 00:00:00,temp,51
1,2022-01-01 01:00:00,windchill,33
2,2022-01-01 02:00:00,temp,75
3,2022-01-01 03:00:00,windchill,40
4,2022-01-01 04:00:00,windchill,96
...,...,...,...
884,2022-02-06 20:00:00,humidity,52
885,2022-02-06 21:00:00,humidity,18
886,2022-02-06 22:00:00,windchill,90
887,2022-02-06 23:00:00,humidity,29


In [27]:
rotated = df.pivot(index='Time', columns='Weather', values='Values')
rotated

Weather,humidity,temp,windchill
Time,,,
2022-01-01 00:00:00,NaN,51.0,NaN
2022-01-01 01:00:00,NaN,NaN,33.0
2022-01-01 02:00:00,NaN,75.0,NaN
2022-01-01 03:00:00,NaN,NaN,40.0
2022-01-01 04:00:00,NaN,NaN,96.0
...,...,...,...
2022-02-06 20:00:00,52.0,NaN,NaN
2022-02-06 21:00:00,18.0,NaN,NaN
2022-02-06 22:00:00,NaN,NaN,90.0


### Resampling time series data.

Resampling data sometimes comes up, though not very often.  Note above we created a dataframe with timeseries data in it, but note that the data is rather spares. We have hourly measurements, but never from more than one sensor and only once an hour. What if we wanted to resample this to show a more dense view?


In [31]:
rotated.resample('D').mean()

Weather,humidity,temp,windchill
Time,,,
2022-01-01,59.800000,38.900000,57.500000
2022-01-02,42.000000,39.714286,58.166667
2022-01-03,35.200000,65.800000,55.714286
2022-01-04,59.777778,31.000000,46.300000
2022-01-05,47.545455,47.000000,56.125000
2022-01-06,44.800000,49.666667,40.857143
2022-01-07,28.400000,37.833333,48.750000
2022-01-08,56.125000,46.888889,70.428571
2022-01-09,49.333333,53.625000,59.900000


Above, we asked pandas to resample this dataset so we got daily average of the time series data, rather than that sparse set of data.  We didnt need hourly data, not really, not in this example, but getting a daily average was good enough.

It is possible to get more complicated though, if thats what you want.

In [30]:
rotated.resample('D').agg(['min', 'max', 'mean', 'std'])

Weather    humidity                              temp                   \
                min   max       mean        std   min   max       mean   
Time                                                                     
2022-01-01      0.0  87.0  59.800000  27.635726   8.0  75.0  38.900000   
2022-01-02      7.0  90.0  42.000000  31.249000   7.0  71.0  39.714286   
2022-01-03      8.0  68.0  35.200000  26.224035  42.0  86.0  65.800000   
2022-01-04     20.0  99.0  59.777778  24.524364   4.0  88.0  31.000000   
2022-01-05      8.0  87.0  47.545455  27.890370  10.0  89.0  47.000000   
2022-01-06     24.0  81.0  44.800000  21.718656   1.0  98.0  49.666667   
2022-01-07      3.0  55.0  28.400000  20.369912   8.0  64.0  37.833333   
2022-01-08     16.0  99.0  56.125000  30.390024   0.0  98.0  46.888889   
2022-01-09      3.0  89.0  49.333333  30.715903  16.0  80.0  53.625000   
2022-01-10      7.0  99.0  57.875000  34.774529  10.0  98.0  42.875000   
2022-01-11      1.0  99.0  43.125000  34.852495  17.0  89.0  46.125000   
2022-01-12      8.0  65.0  40.545455  22.818254   6.0  96.0  66.125000   
2022-01-13     28.0  99.0  63.111111  27.369894  39.0  92.0  65.250000   
2022-01-14     17.0  99.0  62.714286  28.581546  18.0  94.0  65.750000   
2022-01-15      8.0  77.0  49.166667  23.659389   3.0  96.0  44.166667   
2022-01-16      1.0  99.0  56.222222  34.175202  28.0  90.0  68.200000   
2022-01-17     22.0  94.0  62.888889  24.967201  11.0  81.0  56.888889   
2022-01-18     11.0  72.0  42.857143  23.926475  17.0  91.0  59.500000   
2022-01-19      0.0  97.0  48.125000  32.843514  16.0  83.0  54.000000   
2022-01-20      8.0  98.0  53.454545  28.079757  12.0  87.0  50.571429   
2022-01-21     15.0  95.0  54.444444  29.875203  13.0  81.0  42.428571   
2022-01-22      2.0  97.0  49.142857  35.063887  18.0  98.0  55.777778   
2022-01-23      3.0  96.0  48.666667  39.878148   1.0  80.0  47.100000   
2022-01-24      0.0  84.0  30.375000  27.197361   3.0  70.0  37.800000   
2022-01-25     12.0  86.0  49.285714  26.942620  56.0  77.0  67.500000   
2022-01-26     10.0  87.0  39.875000  29.107866   8.0  71.0  37.250000   
2022-01-27      3.0  98.0  50.636364  36.216772  36.0  98.0  56.200000   
2022-01-28      0.0  95.0  33.000000  30.232433   1.0  96.0  42.666667   
2022-01-29      7.0  98.0  47.444444  28.373011   2.0  68.0  28.454545   
2022-01-30      9.0  94.0  50.111111  32.887857   6.0  97.0  54.857143   
2022-01-31     18.0  91.0  62.285714  27.010580  14.0  77.0  44.142857   
2022-02-01      4.0  99.0  43.375000  34.846346  18.0  82.0  58.000000   
2022-02-02     15.0  98.0  51.000000  28.993103  17.0  79.0  47.833333   
2022-02-03     39.0  98.0  72.250000  27.741365   0.0  95.0  53.285714   
2022-02-04     11.0  94.0  50.500000  30.687364   8.0  99.0  46.300000   
2022-02-05      7.0  96.0  40.857143  33.879338  17.0  94.0  58.125000   
2022-02-06     18.0  80.0  44.500000  25.406692  13.0  75.0  50.800000   
2022-02-07      NaN   NaN        NaN        NaN   NaN   NaN        NaN   

Weather               windchill                              
                  std       min   max       mean        std  
Time                                                         
2022-01-01  23.923257      33.0  96.0  57.500000  28.290163  
2022-01-02  22.178926      16.0  88.0  58.166667  24.248086  
2022-01-03  15.786070       8.0  97.0  55.714286  27.594393  
2022-01-04  33.518652       7.0  96.0  46.300000  29.386505  
2022-01-05  36.448594      18.0  94.0  56.125000  33.060280  
2022-01-06  33.232879       4.0  92.0  40.857143  33.358800  
2022-01-07  21.132124       9.0  95.0  48.750000  30.363277  
2022-01-08  38.650499      41.0  87.0  70.428571  18.310809  
2022-01-09  18.844761       3.0  99.0  59.900000  31.423452  
2022-01-10  31.498016       2.0  77.0  42.625000  29.683750  
2022-01-11  25.255480       4.0  98.0  45.250000  37.178143  
2022-01-12  31.674404      64.0  92.0  74.200000  11.189281  
2022-01-13  20.561754       3.0  85